In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from upsetplot import generate_data, plot
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import numpy as np
import seaborn as sns

%matplotlib inline

df = pd.read_csv('atussum_0321.csv', header='infer')
df = df.drop(columns=['GEMETSTA','TEMJOT','TUFNWGTP','TU20FWGT'])

df.head(1)

,TUCASEID,GTMETSTA,PEEDUCA,PEHSPNON,PTDTRACE,TEAGE,TELFS,TESCHENR,TESCHLVL,TESEX,...,t181801,t181899,t189999,t500101,t500103,t500104,t500105,t500106,t500107,t509989
0,20030100013280,-1,44,2,2,60,2,-1,-1,1,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df['TESPEMPNOT'].unique()

array([ 2,  1, -1])

In [3]:
import numpy as np
df = df.rename(columns={'GTMETSTA': 'MetStat', 'TEAGE': 'Age', 'TESEX': 'Sex', 'TRCHILDNUM': 'ChildNum', 'PEEDUCA': 'EduStat', 'PEHSPNON': 'Hisp', 'PTDTRACE': 'Race', 'TELFS': 'LaborStat', 'TESCHENR': 'IsStudying', 'TESCHLVL': 'SchoolLevel', 'TESPEMPNOT': 'IsSpouseEmpl', 'TRSPFTPT': 'SpouseEmplTime', 'TRDPFTPT': 'SelfEmplTime', 'TRERNWA': 'Earnings', 'TRSPPRES': 'HasSpouse', 'TRYHHCHILD': 'YoungestChildAge', 'TEHRUSLT': 'HoursWorked'})

#categorize as per age - derived
def ageCondition(x):
    if x > 15 and x < 20: return "Teenage"
    elif x >= 20 and x < 40: return "Young"
    elif x >= 40 and x < 60: return "Adult"
    elif x >= 60 and x < 85: return "Senior Citizen"
    else: return np.nan
df['AgeGrp'] = df['Age'].apply(ageCondition)

def sexCondition(x):
    if x == 1: return "M"
    elif x == 2: return "F"
    else: return np.nan
df['SexGrp'] = df['Sex'].apply(sexCondition)

def metStatFCondition(x):
    if x < 1 or x > 2: return np.nan
    else: return x
df['MetStat'] = df['MetStat'].map(metStatFCondition)
def metStatCondition(x):
    if x == 1: return "Metro"
    elif x == 2: return "NonMetro"
    else: return np.nan
df['MetroGrp'] = df['MetStat'].apply(metStatCondition)

def eduCondtion(x):
    if x >=31 and x <= 36: return "10th grade"
    if x > 36 and x <= 40: return "High School"
    if x > 40 and x <= 42: return "Associate"
    if x == 43: return "Bachelors"
    if x == 44: return "Masters"
    if x > 44 and x <= 46: return "MD/PhD"
    else: return np.nan
df['EduGrp'] = df['EduStat'].apply(eduCondtion)    

def hispanicCondtion(x):
    if x == 1: return "Hispanic"
    elif x == 2: return "Non-Hispanic"
    else: return np.nan
df['HispGrp'] = df['Hisp'].apply(hispanicCondtion)    

def raceFCondition(x):
    if x !=1 and x != 2 and x != 4: return np.nan
    else: return x
df['Race'] = df['Race'].map(raceFCondition)
def raceCondition(x):
    if x == 1: return "White"
    if x == 2: return "Blacks"
    if x == 4: return "Asians"
    else: return np.nan
df['RaceGrp'] = df['Race'].apply(raceCondition)

def laborFCondition(x):
    if x == 5: return np.nan
    else: return x
df['LaborStat'] = df['LaborStat'].map(laborFCondition)
def laborCondition(x):
    if x == 1 or x == 2: return "Employed"
    elif x == 3 or x == 4: return "Unemployed"
    else: return np.nan
df['LaborGrp'] = df['LaborStat'].apply(laborCondition)

def schoolFCondition(x):
    if x !=1 and x != 2: return np.nan
    else: return x
df['IsStudying'] = df['IsStudying'].map(schoolFCondition)
def schoolCondition(x):
    if x == 1: return "Y"
    elif x == 2: return "N"
    else: return np.nan
df['IsStudyGrp'] = df['IsStudying'].apply(schoolCondition)    
    
def schoolLevelFCondition(x):
    if x !=1 and x != 2: return np.nan
    else: return x
df['SchoolLevel'] = df['SchoolLevel'].map(schoolLevelFCondition)
def schoolLevelCondition(x):
    if x == 1: return "H School"
    elif x == 2:return "College/University"
    else: return np.nan
df['SchoolLevelGrp'] = df['SchoolLevel'].apply(schoolLevelCondition)

def hasSpouseCondition(x):
    if x == 1 or x == 2: return "Y"
    elif x == 3: return "N"
    else: return np.nan
df['SpousePresentGrp'] = df['HasSpouse'].apply(hasSpouseCondition)

def spouseEmployedFCondition(x):
    if x != 1 and x != 2: return np.nan
    else: return x
df['IsSpouseEmpl'] = df['IsSpouseEmpl'].map(spouseEmployedFCondition)
def spouseEmployedCondition(x):
    if x == 1: return "Y"
    elif x == 2: return "N"
    else: return "N"
df['SpouseEmplGrp'] = df['IsSpouseEmpl'].apply(spouseEmployedCondition)

def employmentTimeFCondition(x):
    if x !=1 and x != 2: return np.nan
    else: return x
df['SelfEmplTime'] = df['SelfEmplTime'].map(employmentTimeFCondition)
df['SpouseEmplTime'] = df['SpouseEmplTime'].map(employmentTimeFCondition)
def employmentTimeCondition(x):
    if x == 1: return "FT"
    elif x == 2: return "PT"
    else: return np.nan
df['SelfEmplTimeGrp'] = df['SelfEmplTime'].apply(employmentTimeCondition)
df['SpouseEmplTimeGrp'] = df['SpouseEmplTime'].apply(employmentTimeCondition)

def childrenCondition(x):
    if x == 0: return "None"
    elif x > 0 and x <= 2: return "1-2 children"
    elif x > 2 and x <= 4: return "3-4 children"
    elif x >4 and x <= 30: return ">5 children"
    else: return np.nan
df['ChildNumGrp'] = df['ChildNum'].apply(childrenCondition)

def youngestChildAgeCondition(x):
    if x >= 0 and x <= 3: return "Baby"
    elif x > 3 and x <= 8: return "Kid"
    elif x > 8 and x <= 13: return "GrownKid"
    elif x > 13 and x <= 17: return "Teen"
    else: return np.nan
df['YChildAgeGrp'] = df['YoungestChildAge'].apply(youngestChildAgeCondition)

def earningsCondition(x):
    if x > 0 and x <=50000: return "Low Income"
    elif x > 50000 and x <= 100000: return "Low Middle Class"
    elif x > 100000 and x <= 150000: return "High Middle Class"
    elif x > 150000: return "High Income"
    else: return np.nan
df['EarningsGrp'] = df['Earnings'].apply(earningsCondition)    
    
    
# defining categorical columns
df['AgeGrp'] = pd.Categorical(df['AgeGrp'], 
                                           categories=['Teenage', 'Young', 'Adult', 'Senior Citizen'])
df['SexGrp'] = pd.Categorical(df['SexGrp'], 
                                           categories=['M', 'F'])
df['MetroGrp'] = pd.Categorical(df['MetroGrp'], 
                                           categories=['Metro', 'NonMetro'])
df['EduGrp'] = pd.Categorical(df['EduGrp'], 
                                           categories=['10th grade', 'High School', 'Associate', 'Bachelors', 'Masters', 'MD/PhD'])
df['HispGrp'] = pd.Categorical(df['HispGrp'], 
                                           categories=['Hispanic', 'Non-Hispanic'])
df['RaceGrp'] = pd.Categorical(df['RaceGrp'], 
                                           categories=['White', 'Blacks', 'Asians'])
df['LaborGrp'] = pd.Categorical(df['LaborGrp'], 
                                           categories=['Employed', 'Unemployed'])
df['IsStudyGrp'] = pd.Categorical(df['IsStudyGrp'], 
                                           categories=['Y', 'N'])
df['SchoolLevelGrp'] = pd.Categorical(df['SchoolLevelGrp'], 
                                           categories=['H School', 'College/University'])
df['SpousePresentGrp'] = pd.Categorical(df['SpousePresentGrp'], 
                                           categories=['Y', 'N'])
df['SpouseEmplGrp'] = pd.Categorical(df['SpouseEmplGrp'], 
                                           categories=['Y', 'N'])
df['SpouseEmplTimeGrp'] = pd.Categorical(df['SpouseEmplTimeGrp'], 
                                           categories=['FT', 'PT'])
df['SelfEmplTimeGrp'] = pd.Categorical(df['SelfEmplTimeGrp'], 
                                           categories=['FT', 'PT'])
df['ChildNumGrp'] = pd.Categorical(df['ChildNumGrp'], 
                                           categories=['None', '1-2 children', '3-4 children', '>5 children'])
df['YChildAgeGrp'] = pd.Categorical(df['YChildAgeGrp'], 
                                           categories=['Baby', 'Kid', 'Grown Kid', 'Teen'])
df['EarningsGrp'] = pd.Categorical(df['EarningsGrp'], 
                                           categories=['Low Income', 'Low Middle Class', 'High Middle Class', 'High Income'])

df.head(10)

,TUCASEID,MetStat,EduStat,Hisp,Race,Age,LaborStat,IsStudying,SchoolLevel,Sex,...,LaborGrp,IsStudyGrp,SchoolLevelGrp,SpousePresentGrp,SpouseEmplGrp,SelfEmplTimeGrp,SpouseEmplTimeGrp,ChildNumGrp,YChildAgeGrp,EarningsGrp
0,20030100013280,NaN,44,2,2.0,60,2.0,NaN,NaN,1,...,Employed,NaN,NaN,Y,N,PT,NaN,None,NaN,Low Middle Class
1,20030100013344,NaN,40,2,1.0,41,1.0,2.0,NaN,2,...,Employed,N,NaN,Y,Y,PT,FT,1-2 children,Baby,Low Income
2,20030100013352,NaN,41,2,1.0,26,2.0,2.0,NaN,2,...,Employed,N,NaN,Y,N,PT,NaN,None,NaN,Low Income
3,20030100013848,NaN,39,2,2.0,36,4.0,2.0,NaN,2,...,Unemployed,N,NaN,Y,Y,NaN,FT,1-2 children,NaN,NaN
4,20030100014165,NaN,45,2,1.0,51,1.0,NaN,NaN,1,...,Employed,NaN,NaN,Y,N,FT,NaN,1-2 children,Teen,NaN
5,20030100014169,NaN,43,2,1.0,32,2.0,1.0,2.0,2,...,Employed,Y,College/University,Y,Y,FT,FT,1-2 children,Baby,Low Middle Class
6,20030100014209,NaN,39,2,1.0,44,1.0,2.0,NaN,2,...,Employed,N,NaN,Y,Y,FT,FT,1-2 children,NaN,NaN
7,20030100014427,NaN,40,2,1.0,21,1.0,2.0,NaN,2,...,Employed,N,NaN,N,N,FT,NaN,1-2 children,Teen,Low Income
8,20030100014550,NaN,41,2,1.0,33,1.0,2.0,NaN,2,...,Employed,N,NaN,Y,Y,FT,FT,3-4 children,Baby,Low Middle Class
9,20030100014758,NaN,41,2,2.0,39,1.0,2.0,NaN,2,...,Employed,N,NaN,Y,N,FT,NaN,3-4 children,Kid,Low Income


In [4]:
df['Sex'].isnull().values.any()
df['IsSpouseEmpl'].isnull().values.any()
df['MetroGrp'].

SyntaxError: invalid syntax (2721734350.py, line 3)

In [ ]:
print(df['AgeGrp'].unique())
print(df['SexGrp'].unique())
print(df['MetroGrp'].unique())

In [ ]:
df.to_csv('processedDF.csv')